In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from geopy.distance import great_circle

import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable
from torchsummary import summary
import datetime

import os
import random

# Setting

In [2]:
#  predition TI of leading time at 24 hours
pre_seq = 4
batch_size = 128
epochs = 128
min_val_loss = 100
model_name = '3D_SAF_Net(8).pkl'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
train = pd.read_csv('../../data/CMA_train_'+str(pre_seq*6)+'h.csv', header=None)
test= pd.read_csv('../../data/CMA_test_'+str(pre_seq*6)+'h.csv', header=None)

In [4]:
train.shape, test.shape

((8406, 101), (2747, 101))

In [5]:
CLIPER_feature =  pd.concat((train, test), axis=0)
CLIPER_feature.reset_index(drop=True, inplace=True)

In [6]:
X_wide_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_wide = X_wide_scaler.fit_transform(CLIPER_feature.iloc[:, 5:])
X_wide_train = X_wide[0: train.shape[0], :]

y = y_scaler.fit_transform(CLIPER_feature.loc[:, 3].values.reshape(-1,1))
y_train = y[0: train.shape[0], :]

In [7]:
# now 6 hours ago  12 hours ago  18 hour ago
ahead_times = [0,1,2,3]

pressures = [1000, 900, 800, 700, 600, 500, 400, 300]

sequential_reanalysis_u_list = []
reanalysis_u_test_dict = {}
X_deep_u_scaler_dict = {}

sequential_reanalysis_v_list = []
reanalysis_v_test_dict = {}
X_deep_v_scaler_dict = {}

In [8]:
reanalysis_type = 'u'
for ahead_time in ahead_times:

    reanalysis_list = []
    for pressure in pressures:
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)
            
        train_reanalysis_csv = pd.read_csv('../../data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_train_31_31.csv', header=None)
        test_reanalysis_csv = pd.read_csv('../../data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_test_31_31.csv', header=None)

        train_reanalysis = train_reanalysis_csv[train_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = test_reanalysis_csv[test_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_u_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis # 保存test 用于后面测试
        
        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)

        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_u_scaler_dict[scaler_name] = MinMaxScaler()
        
        # 5:end is the 31*31 u component wind speed
        X_deep = X_deep_u_scaler_dict[scaler_name].fit_transform(reanalysis.loc[:, 5:])
        
         # (batch, type, channel, height, widht, time) here type is u
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 31, 31, 1)
        reanalysis_list.append(X_deep_final)
        
    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_u_list.append(X_deep_temp)

X_deep_u_train = np.concatenate(sequential_reanalysis_u_list, axis=5)

ahead_time: 0 (8406, 1, 8, 31, 31, 1)
ahead_time: 1 (8406, 1, 8, 31, 31, 1)
ahead_time: 2 (8406, 1, 8, 31, 31, 1)
ahead_time: 3 (8406, 1, 8, 31, 31, 1)


In [9]:
reanalysis_type = 'v'
for ahead_time in ahead_times:

    reanalysis_list = []
    for pressure in pressures:
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)

        train_reanalysis_csv = pd.read_csv('../../data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_train_31_31.csv', header=None)
        test_reanalysis_csv = pd.read_csv('../../data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_test_31_31.csv', header=None)

        train_reanalysis = train_reanalysis_csv[train_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = test_reanalysis_csv[test_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_v_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis # 保存test 用于后面测试

        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)

        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_v_scaler_dict[scaler_name] = MinMaxScaler()
        
        # 5:end is the 31*31 u component wind speed
        X_deep = X_deep_v_scaler_dict[scaler_name].fit_transform(reanalysis.loc[:, 5:])
        # (batch, type, channel, height, widht, time) here type is u
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 31, 31, 1)
        reanalysis_list.append(X_deep_final)
        
    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_v_list.append(X_deep_temp)

X_deep_v_train = np.concatenate(sequential_reanalysis_v_list, axis=5)

ahead_time: 0 (8406, 1, 8, 31, 31, 1)
ahead_time: 1 (8406, 1, 8, 31, 31, 1)
ahead_time: 2 (8406, 1, 8, 31, 31, 1)
ahead_time: 3 (8406, 1, 8, 31, 31, 1)


In [10]:
X_deep_train = np.concatenate((X_deep_u_train, X_deep_v_train), axis=1)

In [11]:
X_wide_train.shape, X_deep_train.shape

((8406, 96), (8406, 2, 8, 31, 31, 4))

# training set and validation set

In [12]:
class TrainLoader(Data.Dataset):
    def __init__(self, X_wide_train, X_deep_train, y_train):
        self.X_wide_train = X_wide_train
        self.X_deep_train = X_deep_train
        self.y_train = y_train
        
    def __getitem__(self, index):
        return [self.X_wide_train[index], self.X_deep_train[index]], self.y_train[index]
    
    def __len__(self):
        return len(self.X_wide_train)

In [13]:
full_train_index = [*range(0, len(X_wide_train))]

train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)

In [14]:
len(train_index), len(val_index)

(7565, 841)

In [15]:
train_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size, shuffle=True)

In [16]:
val_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size, shuffle=True)

# 3D SAF-Net

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(2, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv2 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.fc1 = nn.Linear(128 * 7 * 7 * 2, 128)
        self.fc2 = nn.Linear(96 + 128*4, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, wide, deep):
        seq_list = []
        for i in range(len(ahead_times)):
            timeseq = deep[...,i]
            timeseq = self.pool1(F.relu(self.conv1(timeseq)))
            timeseq = self.pool2(F.relu(self.conv2(timeseq)))
            timeseq = timeseq.view(-1, 128 * 7 * 7 * 2)
            timeseq = self.fc1(timeseq)
            seq_list.append(timeseq)
        wide = wide.view(-1, 96)
        wide_n_deep = torch.cat((wide, seq_list[0]),1)
        if len(ahead_times) > 1:
            for i in range(1, len(ahead_times)):
                wide_n_deep = torch.cat((wide_n_deep, seq_list[i]),1)
        wide_n_deep = self.fc2(wide_n_deep)
        wide_n_deep = self.fc3(wide_n_deep)
        wide_n_deep = self.fc4(wide_n_deep)
        return wide_n_deep

net = Net()

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

net

Net(
  (conv1): Conv3d(2, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=608, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)

# training

In [19]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [19]:
full_train_index = [*range(0, len(X_wide_train))]

for epoch in range(epochs):  # loop over the dataset multiple times
    starttime = datetime.datetime.now()
    train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)
    train_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size,)
    val_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size,)
    total_train_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_dataset):
        if torch.cuda.is_available():
            net.cuda()
            X_wide_train_cuda = batch_x[0].float().cuda()
            X_deep_train_cuda = batch_x[1].float().cuda()
            y_train_cuda = batch_y.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_y = net(X_wide_train_cuda, X_deep_train_cuda)
        loss = criterion(pred_y, y_train_cuda)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    total_val_loss = 0
    for _,(batch_val_x, batch_val_y) in enumerate(val_dataset):
        
        if torch.cuda.is_available():
            X_wide_val_cuda = batch_val_x[0].float().cuda()
            X_deep_val_cuda = batch_val_x[1].float().cuda()
            y_val_cuda = batch_val_y.cuda()
        
        pred_y = net(X_wide_val_cuda, X_deep_val_cuda)
        val_loss = criterion(pred_y, y_val_cuda)
        total_val_loss += val_loss.item()
    
        # print statistics
    if min_val_loss > total_val_loss:
        torch.save(net.state_dict(), model_name)
        min_val_loss = total_val_loss
    endtime = datetime.datetime.now()
    print('epochs [%d/%d] cost:%.2fs train_loss: %.5f val_loss: %.5f' % 
          (epoch + 1, epochs, (endtime-starttime).seconds, total_train_loss, total_val_loss))

print('Finished Training')

epochs [1/128] cost:8.00s train_loss: 12.40180 val_loss: 0.52767
epochs [2/128] cost:8.00s train_loss: 4.45969 val_loss: 0.51595
epochs [3/128] cost:8.00s train_loss: 4.33298 val_loss: 0.48625
epochs [4/128] cost:8.00s train_loss: 4.18216 val_loss: 0.48746
epochs [5/128] cost:8.00s train_loss: 4.20319 val_loss: 0.47350
epochs [6/128] cost:8.00s train_loss: 3.97236 val_loss: 0.46013
epochs [7/128] cost:8.00s train_loss: 4.03655 val_loss: 0.45484
epochs [8/128] cost:8.00s train_loss: 3.82168 val_loss: 0.55231
epochs [9/128] cost:8.00s train_loss: 3.87506 val_loss: 0.49459
epochs [10/128] cost:8.00s train_loss: 4.29568 val_loss: 0.44673
epochs [11/128] cost:8.00s train_loss: 3.80378 val_loss: 0.43510
epochs [12/128] cost:8.00s train_loss: 3.69335 val_loss: 0.41612
epochs [13/128] cost:8.00s train_loss: 3.81514 val_loss: 0.42895
epochs [14/128] cost:8.00s train_loss: 3.90686 val_loss: 0.48735
epochs [15/128] cost:8.00s train_loss: 3.75642 val_loss: 0.42843
epochs [16/128] cost:8.00s train_

epochs [127/128] cost:8.00s train_loss: 2.81850 val_loss: 0.31414
epochs [128/128] cost:8.00s train_loss: 2.77843 val_loss: 0.31296
Finished Training


# testing

In [20]:
net.load_state_dict(torch.load(model_name))

years = test[4].unique()

test_list = []

for year in years:
    temp = test[test[4]==year]
    temp = temp.reset_index(drop=True)
    test_list.append(temp)
    
len(test_list)

4

In [21]:
with torch.no_grad():
    for year, _test in zip(years, test_list):

        print(year, '年:')
        y_test = _test.loc[:,3]
        X_wide_test = X_wide_scaler.transform(_test.loc[:,5:])

        final_test_u_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = 'u' +str(pressure) + str(ahead_time)
                X_deep = reanalysis_u_test_dict[scaler_name][reanalysis_u_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_u_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 31, 31, 1)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_u_list.append(X_deep_temp)
        X_deep_u_test = np.concatenate(final_test_u_list, axis=5)
        
        final_test_v_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = 'v' +str(pressure) + str(ahead_time)
                X_deep = reanalysis_v_test_dict[scaler_name][reanalysis_v_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_v_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 31, 31, 1)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_v_list.append(X_deep_temp)
        X_deep_v_test = np.concatenate(final_test_v_list, axis=5)
    
        X_deep_test = np.concatenate((X_deep_u_test, X_deep_v_test), axis=1)
        
        if torch.cuda.is_available():
            X_wide_test = Variable(torch.from_numpy(X_wide_test).float().cuda())
            X_deep_test = Variable(torch.from_numpy(X_deep_test).float().cuda())

        pred = net(X_wide_test, X_deep_test)

        pred = y_scaler.inverse_transform(pred.cpu().detach().numpy().reshape(-1,1))
        true = y_test.values.reshape(-1, 1)
        diff = np.abs(pred - true)

        print('avg wind error:', sum(diff)/len(diff))

2015 年:
avg wind error: [4.7979302]
2016 年:
avg wind error: [6.31414002]
2017 年:
avg wind error: [4.34779083]
2018 年:
avg wind error: [5.2558387]
